In [1]:
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.parse
import time

In [2]:
def get_goodreads_to_read_list(url):
    # Set up Selenium WebDriver (using Chrome in this example)
    driver = webdriver.Chrome()

    # Open the Goodreads page
    driver.get(url)

    # Wait for the page to load completely (you might need to adjust the time or use WebDriverWait for better control)
    driver.implicitly_wait(10)

    # Get the page source after the content has been loaded
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Close the driver
    driver.quit()

    # Find all book titles
    books = [book.get_text(strip=True) for book in soup.select('td.field.title div.value a')]

    return books

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_goodreads_to_read_list(url):
    # Set up Selenium WebDriver (using Chrome in this example)
    driver = webdriver.Chrome()

    # Open the Goodreads page
    driver.get(url)

    # Initialize an empty list to store all book titles
    all_books = []

    while True:
        # Wait for the page to load completely
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'td.field.title div.value a'))
            )
        except Exception as e:
            print(f"Error waiting for page to load: {e}")
            break
        
        # Get the page source after the content has been loaded
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract books from the current page
        books = [book.get_text(strip=True) for book in soup.select('td.field.title div.value a')]
        all_books.extend(books)
        
        # Scroll to the bottom of the page to ensure the "Next" button is in view
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Wait for the page to adjust
        
        # Try to find the "Next" button
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'a.next_page'))
            )
            
            # Debugging: Print the class of the "Next" button
            print(f"Next button class: {next_button.get_attribute('class')}")
            
            if "disabled" in next_button.get_attribute("class"):
                print("No more pages to load.")
                break  # Exit the loop if "Next" button is disabled
            
            # Click on the "Next" button to go to the next page
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)  # Wait for the next page to load
        except Exception as e:
            print(f"Error finding or clicking the 'Next' button: {e}")
            break  # Exit the loop if the "Next" button is not found or clickable

    # Close the driver
    driver.quit()

    return all_books


In [18]:
# Replace with your Goodreads "Want to Read" shelf URL
#goodreads_url = "https://www.goodreads.com/review/list/32680117?shelf=to-read"
#to_read_list = get_goodreads_to_read_list(goodreads_url)
#print(to_read_list)

Next button class: next_page
Next button class: next_page
Next button class: next_page
Next button class: next_page
Next button class: next_page
Next button class: next_page
Next button class: next_page
Error finding or clicking the 'Next' button: Message: 
Stacktrace:
0   chromedriver                        0x0000000104f09024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000104f01700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000104b1082c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104b54834 cxxbridge1$string$len + 367060
4   chromedriver                        0x0000000104b8c48c cxxbridge1$string$len + 595500
5   chromedriver                        0x0000000104b49474 cxxbridge1$string$len + 321044
6   chromedriver                        0x0000000104b4a0e4 cxxbridge1$string$len + 324228
7   chromedriver                        0x0000000104ed0a08 cxxbridge1$str$ptr + 1656336
8   chromedriver 

In [21]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def search_deslegte_book(book_title):
    # Encode book title for URL
    search_url = "https://www.deslegte.com/boeken/?q=" + urllib.parse.quote_plus(book_title)
    print(f"Searching for book: {search_url}")
    
    # Fetch the search results page
    response = requests.get(search_url)
    #print(f"HTTP Status Code: {response.status_code}")
    
    # Check if the request was successful
    if response.status_code != 200:
        print("Failed to retrieve the search results page.")
        return False

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Debug: Print out the page content to inspect it
    # print("Page Content Preview:")
    # print(soup.prettify()[:1000])  # Print the first 1000 characters for inspection
    
    # Extract the number of results from the meta description
    meta_description = soup.find('meta', {'name': 'description'})
    if meta_description:
        description_content = meta_description.get('content', '')        
        if 'uitgaven gevonden' in description_content:
            # Extract the number of editions found
            try:
                number_of_editions = int(description_content.split(' ')[0])
                print(f"Number of Editions Found: {number_of_editions}")
            except ValueError:
                number_of_editions = 0
                print("Error parsing the number of editions.")
            
            if number_of_editions > 0:
                # Extract book titles from search results
                return True
                
            else:
                print(f"No editions found for '{book_title}'.")
                return False
        else:
            print(f"Meta description does not contain 'uitgaven gevonden'.")
            return False
    else:
        print("Meta description tag not found.")
        return False



In [22]:
def check_books_in_deslegte(to_read_list):
    available_books = []
    for book in to_read_list:
        print(f"Checking availability for book: {book}")
        if search_deslegte_book(book):
            available_books.append(book)
    return available_books


In [23]:
# Example to_read_list for testing
#available_books = check_books_in_deslegte(to_read_list)
#print(f"Want To Read books available in Deslegte: {available_books}")

Checking availability for book: It Lasts Forever and Then It's Over
Searching for book: https://www.deslegte.com/boeken/?q=It+Lasts+Forever+and+Then+It%27s+Over
Number of Editions Found: 0
No editions found for 'It Lasts Forever and Then It's Over'.
Checking availability for book: The Complex Answer: On Art as a Non-Binary Intelligence
Searching for book: https://www.deslegte.com/boeken/?q=The+Complex+Answer%3A+On+Art+as+a+Non-Binary+Intelligence
Number of Editions Found: 0
No editions found for 'The Complex Answer: On Art as a Non-Binary Intelligence'.
Checking availability for book: The Structured Vacuum: Thinking About Nothing
Searching for book: https://www.deslegte.com/boeken/?q=The+Structured+Vacuum%3A+Thinking+About+Nothing
Number of Editions Found: 0
No editions found for 'The Structured Vacuum: Thinking About Nothing'.
Checking availability for book: Philosophy of the Home: Domestic Space and Happiness
Searching for book: https://www.deslegte.com/boeken/?q=Philosophy+of+the+Ho

In [24]:
#import pprint

In [25]:
#pprint.pprint(available_books)

['Running',
 'Weapons of Math Destruction: How Big Data Increases Inequality and Threatens '
 'Democracy',
 'Intermezzo',
 'Selected Stories',
 'Correspondence',
 'The Music of Life',
 'Rave',
 'Crash',
 'On Connection',
 'The Blue Fox',
 'Solaris',
 'Salt',
 'The Sailor Who Fell from Grace with the Sea',
 'Het smelt',
 'Bluets',
 'Te intelligent om gelukkig te zijn?',
 'In Praise of Shadows',
 'No Longer Human',
 'Snow Country',
 'Theory of Games and Economic Behavior',
 'Habibi',
 'Medea',
 'The Master and Margarita',
 "The Queen's Gambit",
 'The Stranger',
 'The Age of Spiritual Machines',
 'Kafka on the Shore',
 'Alone With You in the Ether',
 'A Brief History of Time',
 'East of Eden',
 'Gravity’s Rainbow',
 'The Singularity is Near: When Humans Transcend Biology',
 'Manna',
 'Something Deeply Hidden: Quantum Worlds and the Emergence of Spacetime',
 'The Art of War',
 'The Idiot',
 'The Double',
 'Notes from Underground',
 'Dead Souls',
 'A Confession',
 'The Art of Memory']
